<a href="https://colab.research.google.com/github/from1pandas-importLove/Market_Agents_using_FinRL_python/blob/master/FinRLipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Market agents using FinRL

### Install Dependencies

In [1]:
# Run twice if for some reason the process is stopped

!pip install wrds
!pip install swig
!pip install -q condacolab
import condacolab
condacolab.install()
!apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git
!pip install torch_geometric
!pip install optuna
!pip install streamlit
!pip install quantstats
!pip install pandas-market-calendars

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 26.7 MB/s eta 0:00:00
⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:09
🔁 Restarting kernel...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package swig4.0.
(Reading database ... 126332 files and directories currently installed.)
Preparing to unpack .../swig4.0_4.0.2-1ubuntu1_amd64.deb ...
Unpacking swig4.0 (4.0.2-1ubuntu1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_4.0.2-1ubuntu1_all.deb ...
Unpacking swig (4.0.2-1ubuntu1) ...
Setting up swig4.0 (4.0.2-1ubuntu1) ...
Setting

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 123.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 132.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.30.2
    Uninstalling protobuf-6.30.2:
      Successfully uninstalled protobuf-6.30.2


###Import Libraries

In [1]:
# import librabries and set the device
import torch
import numpy as np
import pandas as pd
from sklearn.preprocessing import MaxAbsScaler
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import GroupByScaler
from finrl.meta.env_portfolio_optimization.env_portfolio_optimization import PortfolioOptimizationEnv
from finrl.agents.portfolio_optimization.models import DRLAgent
from finrl.agents.portfolio_optimization.architectures import EIIE
import optuna

# Set device
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

print(device)


cuda:0


###Stage 1: Preprocess the data

In [2]:
# You can use the follow implementation for the stage 1
# or use the one you implemented on stage 1

# ------------------------------------------------------------------------------------------------

# Define your custom stock list
# EX: "AAPL", "MSFT", "GOOGL", "AMZN"
# Choose the ones, that you want to work with
CUSTOM_STOCK_LIST = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "FB",
    "TSLA", "BRK.B", "JNJ", "V", "WMT"
]


# Download stock data
START_DATE = '2011-01-01' # start date of the portfolio
END_DATE = '2022-12-31' # end  date of the portfolio
portfolio_raw_df = YahooDownloader(start_date=START_DATE,
                                   end_date=END_DATE,
                                   ticker_list=CUSTOM_STOCK_LIST).fetch_data()

# Group by ticker and count occurrences
portfolio_raw_df.groupby("tic").count()

# Normalize the data
# You can use GroupByScaler with a MaxAbsScaler here
portfolio_norm_df = GroupByScaler(by="tic", scaler=MaxAbsScaler).fit_transform(portfolio_raw_df)

# Select relevant columns
df_portfolio = portfolio_norm_df[["date", "tic", "close", "high", "low"]]

# Split data into training and testing sets

START_DATE_TRAIN = "2011-01-01" # you start date for the train data
END_DATE_TRAIN = "2019-12-31" # your end date for the train data
START_DATE_TEST = "2020-01-01" # your start date for the test data
END_DATE_TEST = "2022-12-31" # your end date for the test data

df_portfolio_train = df_portfolio[(df_portfolio["date"] >= START_DATE_TRAIN) & (df_portfolio["date"] < END_DATE_TRAIN)]
df_portfolio_test = df_portfolio[(df_portfolio["date"] >= START_DATE_TEST) & (df_portfolio["date"] < END_DATE_TEST)]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['FB']: YFTzMissingError('possibly delisted; no timezone found')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['BRK.B']: YFTzMissingError('possibly delisted; no timezone found')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (24160, 8)



/usr/local/lib/python3.11/site-packages/finrl/meta/preprocessor/preprocessors.py:100: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.23665012 0.16431745 0.13584257 ... 0.04542185 0.04024656 0.0409539 ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X.loc[select_mask, self.columns] = self.scalers[value].transform(
/usr/local/lib/python3.11/site-packages/finrl/meta/preprocessor/preprocessors.py:100: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.   0.25 0.5  ... 0.5  0.75 1.  ]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  X.loc[select_mask, self.columns] = self.scalers[value].transform(


In [3]:
# Print the train and test dfs shape
TRAIN_DF_SHAPE = df_portfolio_train.shape
TEST_DF_SHAPE = df_portfolio_test.shape
print("Train df shape: ", TRAIN_DF_SHAPE)
print("Test df shape: ", TEST_DF_SHAPE)

Train df shape:  (18104, 5)
Test df shape:  (6048, 5)


### Stage 2: Train the model


In [ ]:
# Define the environment
# We will use portfolio optimization for the project
INITIAL_AMOUT = 100000 # initial amount of money in the portfolio: float
COMISSION_FEE_PTC = 0.001 # commission fee: float
TIME_WINDOW = 50# time window: int
FEATURES = ["close", "high", "low"] # ex: "close", "high"

environment_train = PortfolioOptimizationEnv(
    df_portfolio_train,
    initial_amount=INITIAL_AMOUT,
    comission_fee_pct=COMISSION_FEE_PTC,
    time_window=TIME_WINDOW,
    features=FEATURES,
    normalize_df=None # df is already normalized
)

# Set PolicyGradient parameters
# Set the learning rate for the training
model_kwargs = {
    "lr": 0.001, # put a learning rate, ex: 0.01
    "policy": EIIE, # we will use EIIE policy for this project
}

# Set EIIE's parameters
policy_kwargs = {
    "k_size": 3, # put the k_size: int
    "time_window": TIME_WINDOW, # time window defined previously
}

# Instantiate the model
model = DRLAgent(environment_train).get_model("pg", device, model_kwargs, policy_kwargs)

# Train the model
EPISODES = 500 # number of episodes to training the model: in
DRLAgent.train_model(model, episodes=EPISODES)

# Save the model
torch.save(model.train_policy.state_dict(), "policy_EIIE.pt")

  0%|          | 0/500 [00:00<?, ?it/s]

Initial portfolio value:100000
Final portfolio value: 687994.4375
Final accumulative portfolio value: 6.879944324493408
Maximum DrawDown: -0.168190741208595
Sharpe ratio: 1.4936780622557806


findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
  0%|          | 1/500 [00:23<3:16:19, 23.61s/it]

Initial portfolio value:100000
Final portfolio value: 702212.1875
Final accumulative portfolio value: 7.022121906280518
Maximum DrawDown: -0.16758873425847487
Sharpe ratio: 1.4923012156820634


findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
  0%|          | 2/500 [00:46<3:13:34, 23.32s/it]

...

findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
  5%|▍         | 23/500 [08:31<2:54:38, 21.97s/it]

Initial portfolio value:100000
Final portfolio value: 1369686.375
Final accumulative portfolio value: 13.696864128112793
Maximum DrawDown: -0.48329403205512167
Sharpe ratio: 1.0394534880856134


findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
  5%|▍         | 24/500 [08:53<2:54:01, 21.94s/it]

Initial portfolio value:100000
Final portfolio value: 1374370.125
Final accumulative portfolio value: 13.743700981140137
Maximum DrawDown: -0.4869051417721961
Sharpe ratio: 1.0339457947294577


findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
findfont: Font family 'Arial' not found.
  5%|▌         | 25/500 [09:15<2:53:57, 21.97s/it]

In [ ]:
# print the final value of the portfolio
final_portfolio_value_train = environment_train._asset_memory["final"][-1]
print("The final portfolio value at train is:", final_portfolio_value_train)

### evaluate the model

In [ ]:
# Evaluate the model
# Create test env

INITIAL_AMOUT = 100000 # initial amount of money in the portfolio: float
COMISSION_FEE_PTC = 0.001 # commission fee: float
TIME_WINDOW = 50# time window: int
FEATURES = ["close", "high", "low"] # ex: "close", "high"

enviroment_test = PortfolioOptimizationEnv(
    df_portfolio_test,
    initial_amount=INITIAL_AMOUT,
    comission_fee_pct=COMISSION_FEE_PTC,
    time_window=TIME_WINDOW,
    features=FEATURES,
    normalize_df=None # df is already normalized
)


EIIE_results = {
    "train": environment_train._asset_memory["final"],
    "test": {},
}

# instantiate an architecture with the same arguments used in training
# and load with load_state_dict.
policy = EIIE(time_window=TIME_WINDOW, device=device)
policy.load_state_dict(torch.load("policy_EIIE.pt"))

# testing
DRLAgent.DRL_validation(model, enviroment_test, policy=policy)
EIIE_results["test"] = enviroment_test._asset_memory["final"]

In [ ]:
# print the final value of the portfolio
final_portfolio_value_test = enviroment_test._asset_memory["final"][-1]
print("The final portfolio value at test is:", final_portfolio_value_test)